# file path

In [10]:
def filter_contours_by_shape(mask):
    
    # Finding contours 
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an empty mask to store the filtered contours
    leaf_mask = np.zeros_like(mask)

    # contours based on size and shape
    for contour in contours:
        area = cv2.contourArea(contour)

        if area > 1000:  # Filter based on area
            # Approximate the contour to reduce noise
            perimeter = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)

            # Use the number of vertices to estimate the shape (leaf-like shapes are usually smooth)
            if len(approx) > 5:  # Ignore shapes that are too simple (like lines or small blobs)
                cv2.drawContours(leaf_mask, [contour], -1, 255, thickness=cv2.FILLED)

    return leaf_mask

In [11]:
# this function has a similarity with segment_leaf but in case of detecting blights

def detect_blight(image, leaf_mask):
    
    # image to HSV 
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # HSV range for brown/orange blighted leaves
    lower_blighted = np.array([10, 40, 40])
    upper_blighted = np.array([25, 255, 255])

    # mask for blighted regions
    blight_mask = cv2.inRange(hsv, lower_blighted, upper_blighted)

    # Filter the blight mask by the leaf mask (to ensure blights are within the leaf region)
    blight_mask = cv2.bitwise_and(blight_mask, leaf_mask)

    # morphology operation for cleaning
    kernel = np.ones((5, 5), np.uint8)
    blight_mask = cv2.morphologyEx(blight_mask, cv2.MORPH_CLOSE, kernel)  # Close small holes

    return blight_mask

In [12]:
def draw_circles_around_blight(image, blight_mask):

    # Find contours of the blighted regions
    contours, _ = cv2.findContours(blight_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw circles around each blighted region
    for contour in contours:
        # Calculate the minimum enclosing circle for each blighted region
        (x, y), radius = cv2.minEnclosingCircle(contour)

        # Drawing circles on the original image
        center = (int(x), int(y))
        radius = int(radius)

        # Draw a thin circle 
        cv2.circle(image, center, radius, (0, 0, 255), thickness=2)  # Red circles

    return image

def remove_background(image, mask):

    # Convert the mask to a 3-channel image
    mask_3channel = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    # Apply the mask to the original image to keep the leaves only
    result = cv2.bitwise_and(image, mask_3channel)

    return result

In [13]:
def process_images(input_folder, output_folder):
    
    #output folder existence
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop over all images in the folder
    for subdir, _, files in os.walk(input_folder):
        for file in files:
            file_path = os.path.join(subdir, file)
            image = cv2.imread(file_path)

            if image is None:
                print(f"Could not open image: {file_path}")
                continue

            print(f"Processing image: {file_path}")

            # Segment the leaf from the image
            initial_mask = segment_leaf(image)

            # Filter the mask 
            leaf_mask = filter_contours_by_shape(initial_mask)

            # Detect blighted regions
            blight_mask = detect_blight(image, leaf_mask)

            # Draw circles around the blighted regions
            image_with_circles = draw_circles_around_blight(image, blight_mask)

            # Remove the background
            result = remove_background(image_with_circles, leaf_mask)

            # output path
            output_path = os.path.join(output_folder, os.path.relpath(file_path, input_folder))
            output_dir = os.path.dirname(output_path)

            # existence of the output directory exists
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            # Saving the result
            if not cv2.imwrite(output_path, result):
                print(f"Failed to save image: {output_path}")
            else:
                print(f"Processed and saved: {output_path}")

In [14]:
# Example usage:
train_folder = "P:/1-uni/machine-learning/PlantDoc-Dataset/train"
test_folder = "P:/1-uni/machine-learning/PlantDoc-Dataset/test"
output_folder = "P:/1-uni/machine-learning/PlantDoc-Dataset/output"

# Process the train folder
process_images(train_folder, output_folder + "/train")

# Process the test folder
# process_images(test_folder, output_folder + "/test")

Processing image: P:/1-uni/machine-learning/PlantDoc-Dataset/train\Apple leaf\20130519yellowingappleleaves.jpg
Processed and saved: P:/1-uni/machine-learning/PlantDoc-Dataset/output/train\Apple leaf\20130519yellowingappleleaves.jpg
Processing image: P:/1-uni/machine-learning/PlantDoc-Dataset/train\Apple leaf\2017-09-23-07-48-04.jpg
Processed and saved: P:/1-uni/machine-learning/PlantDoc-Dataset/output/train\Apple leaf\2017-09-23-07-48-04.jpg
Processing image: P:/1-uni/machine-learning/PlantDoc-Dataset/train\Apple leaf\Apple-Leaves-Diabetes.jpg
Processed and saved: P:/1-uni/machine-learning/PlantDoc-Dataset/output/train\Apple leaf\Apple-Leaves-Diabetes.jpg
Processing image: P:/1-uni/machine-learning/PlantDoc-Dataset/train\Apple leaf\apple-tree-branch-blossom-plant-fruit-berry-leaf-flower-food-green-produce-evergreen-flora-sad-shrub-apples-branch-with-apples-flowering-plant-rose-family-acerola-malpighia-woody-plant-land-plant-928225.jpg
Processed and saved: P:/1-uni/machine-learning/Plan

KeyboardInterrupt: 